In [1]:
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.nn.functional as F

from nemo.siren import Siren

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# autoreload
%load_ext autoreload
%autoreload 2

# Lunar DEM

In [2]:
img_np = np.load('../data/lunar_dem.npy')

In [ ]:
img_np.shape

In [ ]:
img_np_ds = img_np[::100, ::100]
plt.imshow(img_np_ds)
plt.show()

In [ ]:
# Look at 1000x1000 patch in the top left corner
patch = img_np[:1000, :1000]
Z = patch

# Display the image
plt.imshow(patch)
plt.show()

### Surface plot to scale

In [ ]:
xs = torch.linspace(0, 118500, steps=Z.shape[0], device=device)
ys = torch.linspace(0, 118400, steps=Z.shape[1], device=device)
x, y = torch.meshgrid(xs, ys, indexing='xy')

fig = go.Figure(data=[go.Surface(z=Z, x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

### Scale to -1 to 1

In [ ]:
Z_normalized = 1 * (Z - np.min(Z)) / (np.max(Z) - np.min(Z))   # 0 to 1
xs = torch.linspace(-1, 1, steps=Z.shape[0], device=device)
ys = torch.linspace(-1, 1, steps=Z.shape[1], device=device)
x, y = torch.meshgrid(xs, ys, indexing='xy')
xy = torch.hstack((x.reshape(-1, 1), y.reshape(-1, 1)))

In [ ]:
fig = go.Figure(data=[go.Surface(z=Z_normalized, x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
siren = Siren(in_features=2, 
              out_features=1, 
              hidden_features=256,
              hidden_layers=3, 
              outermost_linear=True,
              first_omega_0=30.0,
              hidden_omega_0=100.0).to(device)

siren.load_state_dict(torch.load('../models/lunar_dem_siren.pth'))
siren.eval()
pass

In [ ]:
with torch.no_grad():
    pred, coords = siren(xy)

In [ ]:
# Plot the predictions
fig = go.Figure(data=[go.Surface(z=pred.cpu().numpy().reshape(Z.shape), x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

# SIREN

### Mt Bruno elevation data

In [ ]:
import pandas as pd

#z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
z_vals = np.load('../data/mt_bruno_elevation.npy')

# Guess the xy scale
xy = 200 * np.mgrid[-12:13, -12:13]
xvals = xy[0]
yvals = xy[1]

# Plot the data
fig = go.Figure(data=[go.Surface(z=z_vals, x=xvals, y=yvals)])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# Scale the XY data to -1 to 1
xy_scaled = xy / 2400
x_scaled = xy_scaled[0]
y_scaled = xy_scaled[1]
z_scaled = z_vals / np.max(z_vals)

In [ ]:
# Plot the scaled data
fig = go.Figure(data=[go.Surface(z=z_scaled, x=x_scaled, y=y_scaled)])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# Fit a Siren network to the data

siren = Siren(in_features=2, out_features=1, hidden_features=256,
                hidden_layers=3, outermost_linear=True).to(device)

In [ ]:
# Train the network

# Loss function
criterion = nn.MSELoss()

# Optimizer
optimizer = torch.optim.Adam(siren.parameters(), lr=1e-5)

# Convert the data to torch tensors
xy_tensor = torch.tensor(xy_scaled, dtype=torch.float32).to(device)
xy_tensor = xy_tensor.reshape(2, -1).T
z_tensor = torch.tensor(z_scaled, dtype=torch.float32).to(device)
z_tensor = z_tensor.reshape(-1, 1)

# Train the network
for step in range(5000):
    # Forward pass
    pred, coords = siren(xy_tensor)

    # Compute loss
    loss = criterion(pred, z_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss every 500 steps
    if step % 500 == 0:
        print(f"Step {step}, Loss {loss.item()}")

In [ ]:
torch.save(siren.state_dict(), '../models/mt_bruno_siren.pt')

In [ ]:
# Sample the Siren network to get the predicted elevation
with torch.no_grad():
    pred, coords = siren(xy_tensor)

# Plot the predictions
fig = go.Figure(data=[go.Surface(z=pred.cpu().numpy().reshape(25, 25), x=x_scaled, y=y_scaled)])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# MSE loss
criterion(pred, z_tensor)

Increase the sampling resolution

In [ ]:
xs = torch.linspace(-1, 1, steps=100, device=device)
ys = torch.linspace(-1, 1, steps=100, device=device)
x, y = torch.meshgrid(xs, ys, indexing='xy')
xy = torch.hstack((x.reshape(-1, 1), y.reshape(-1, 1)))

In [ ]:
with torch.no_grad():
    pred, coords = siren(xy)

# Plot the predictions
fig = go.Figure(data=[go.Surface(z=pred.cpu().numpy().reshape(100, 100), x=x.cpu().numpy(), y=y.cpu().numpy())])
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

# Plan a path over the SIREN

Compute spatial derivatives

In [ ]:
siren = Siren(in_features=2, out_features=1, hidden_features=256,
                hidden_layers=3, outermost_linear=True).to(device)

siren.load_state_dict(torch.load('../models/mt_bruno_siren.pt'))
siren.eval()

In [ ]:
z_pred, coords = siren(xy)

In [ ]:
def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [ ]:
z_xy_grad = gradient(z_pred, coords)

x_grad = z_xy_grad[:, 0].detach().cpu().numpy().reshape(100, 100)
y_grad = z_xy_grad[:, 1].detach().cpu().numpy().reshape(100, 100)

In [ ]:
# Visualize the gradients at 2D heatmaps

from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2, subplot_titles=('X Gradient', 'Y Gradient'))
fig.add_trace(go.Heatmap(z=x_grad), row=1, col=1)
fig.add_trace(go.Heatmap(z=y_grad), row=1, col=2)
fig.update_layout(width=1400, height=700, scene_aspectmode='data')
fig.show()

## Plan a discrete path

from $(-1, 1)$ to $(1, 1)$

We want the path that minimizes distance (number of waypoints) and XY gradient values

In [ ]:
GRID_LEN = 100

# start_idx = (0, 0)                   # (-1, -1)
# end_idx = (GRID_LEN-1, GRID_LEN-1)   # (1, 1)

start_idx = (0, GRID_LEN-1)                   # (-1, -1)
end_idx = (GRID_LEN-1, 0)   # (1, 1)

grad_costmat = (np.abs(x_grad) + np.abs(y_grad))

In [ ]:
# Run A* on the cost matrix
from global_planner import GlobalPlanner

gp = GlobalPlanner(grad_costmat)
path = gp.plan(start_idx, end_idx)

fig, ax = plt.subplots(figsize=(10,10))
gp.plot(ax)
plt.show()

In [ ]:
grad_costmat

In [ ]:
path_xs = xs[path[:,0]]
path_ys = ys[path[:,1]]
path_xy = torch.hstack((path_xs[:,None], path_ys[:,None]))
path_zs, _ = siren(path_xy)

In [ ]:
# Plot path on surface plot
fig = go.Figure()
fig.add_trace(go.Surface(z=pred.cpu().numpy().reshape(100, 100), x=x.cpu().numpy(), y=y.cpu().numpy()))
fig.add_trace(go.Scatter3d(x=path_xs.detach().cpu().numpy(), 
                           y=path_ys.detach().cpu().numpy(), 
                           z=path_zs.detach().cpu().numpy().flatten(), 
                           mode='markers', marker=dict(size=3, color='red')))
fig.update_layout(width=1200, height=700, scene_aspectmode='data')
fig.show()

In [ ]:
# save fig as html
fig.write_html('path.html')

## Plan a continuous path

Parameterize the path with splines